# MLP CIFAR10

Jazon Samillano

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()  # Load cifar10 dataset

170500096/170498071 [==============================] - 11s 0us/step


In [ ]:
# Compute the number of labels.
num_labels = len(np.unique(y_train))

# Convert to one-hot vector.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size * 3

# Resize and normalize.
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255

In [ ]:
# Network parameters.
batch_size = 128
dropout = 0.15

# Model is a 4-layer MLP with ReLU and dropout after each layer.
model = Sequential()
model.add(Dense(1024, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))  # Output for one-hot vector
model.summary()

model.compile(loss='categorical_crossentropy',                               # Loss function for one-hot vector
              optimizer=keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),     # RMSprop optimizer
              metrics=['accuracy'])                                          # Accuracy is good metric for classification tasks.

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)  # Early stop if no longer improving
filepath = 'mlp-cifar10-{epoch:02d}-{val_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')  # Checkpoint

history = model.fit(x_train, y_train, epochs=200, batch_size=batch_size, verbose=1, validation_split=0.2, callbacks=[early_stop, checkpoint])

# Validate the model on test dataset to determine generalization.
_, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 1024)              3146752   
_________________________________________________________________
activation_10 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_11 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)               

In [ ]:
from tensorflow import keras
loaded_cifar10_mlp = keras.models.load_model('mlp-cifar10-78-0.56.hdf5')

In [ ]:
%rm -f mlp-cifar10*

In [ ]:
_, acc = loaded_cifar10_mlp.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))


Test accuracy: 55.2%
